En este sencillo cuaderno, utilizamos una red neuronal totalmente conectada para resolver un problema de clasificación visto anteriormente: el problema de identificación de la física de partículas.

Acompaña al Capítulo 8 del libro.

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman.
Traducido por Lucia Perez y Rosario Cecilio-Flores-Elie. 
Licencia: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 150)

font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
matplotlib.rcParams['figure.dpi'] = 300

Tensorflow es una biblioteca muy utilizada en el desarrollo de modelos de aprendizaje profundos. Es una plataforma de código abierto desarrollada por Google. Admite la programación en varios lenguajes, p. C++, Java, Python y muchos otros.

Keras es una API (interfaz de programación de aplicaciones) de alto nivel que se basa en TensorFlow (o Theano, otra biblioteca de aprendizaje profundo). Es específico de Python y podemos considerarlo como el equivalente de la biblioteca sklearn para redes neuronales. Es menos general y menos personalizable, pero es muy fácil de usar y comparativamente más fácil que TensorFlow. Usaremos keras con el back-end de tensorflow.

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
import keras

from keras.models import Sequential #el modelo se construye agregando capas una tras otra

from keras.layers import Dense #capas totalmente conectadas: cada salida habla con cada entrada

from keras.layers import Dropout #para la regularización

Comenzamos con el problema 4top vs ttbar, y usamos la configuración donde agregamos las características "número de leptones", "número de chorros", etc. Como referencia, el SVM óptimo logró una precisión del 94-95%. Tenga en cuenta que esos números no se habían ejecutado a través de una validación cruzada <b> anidada </b>, por lo que podrían ser ligeramente optimistas.

In [ ]:
X = pd.read_csv('../data/Features_lim_2.csv')

In [ ]:
y = np.genfromtxt('../data/Labels_lim_2.txt')

In [ ]:
X.values.shape

No hay un proceso de validación cruzada "incorporado" (o validación cruzada anidada), por lo que tendríamos que construirlo nosotros mismos. Por ahora, podemos construir tres conjuntos: entrenar, validar (para optimización de parámetros) y probar (para evaluación final). Idealmente, deberíamos construir esto como una estructura de validación cruzada.

In [ ]:
#Siempre barajar primero

X,y = shuffle(X,y, random_state = 10)

In [ ]:
X_train = X.values[:3000,:]
y_train = y[:3000]

In [ ]:
X_val = X.values[3000:4000,:]
y_val = y[3000:4000]

In [ ]:
X_test = X.values[4000:,:]
y_test = y[4000:]

In [ ]:
X_train.shape, X_val.shape, X_test.shape

### Construyendo la red

Pensemos en la arquitectura del modelo.

Nuestra capa de entrada tiene 24 neuronas.

Nuestra capa de salida tiene una neurona (lo que sale es la probabilidad que el objeto pertenezca a la clase positiva). También podríamos configurarlo como dos neuronas (y tener softmax como la no linealidad final), pero esto es redundante en un problema de clasificación binaria.

Agregaremos dos capas ocultas. Aquí estoy haciendo sus tamaños = 20 (¡debería optimizar este hiperparámetro!). También podemos reservar la posibilidad de añadir una capa de dropout después de cada uno. La fracción de abandono ("dropout") también debe optimizarse a través de VC.

Otras decisiones que tenemos que tomar son: qué no linealidades usamos (por ahora: ReLU para capas ocultas, sigmoid para la final), qué optimizador usamos (Adam), qué tasa de aprendizaje inicial adoptamos (aquí 0.001, pero nuevamente esto debe decidirse a través de VC), el número de épocas (por ejemplo, 100; podemos trazar cantidades de interés para verificar que tenemos suficiente), el tamaño del lote para el paso de descenso del gradiente (aquí 200, ¡pero puede explorar!) y la función de pérdida . La última es la entropía cruzada binaria, que es la opción estándar para los problemas de clasificación en los que generamos una probabilidad. Premia la "confianza" en una predicción correcta (alta probabilidad).

Los siguientes comandos se pueden utilizar para explorar estas opciones.

In [ ]:
dir(keras.optimizers)

In [ ]:
dir(keras.losses)

Una opción estándar para un caso como el nuestro, donde las etiquetas son 0/1 pero podemos predecir una probabilidad, es la entropía cruzada binaria o pérdida logarítmica:

L = - $\frac{1}{N} \sum_{i=1}^N y_i \cdot log(p(y_i)) + (1-y_i) \cdot log (1 - p(y_i))$

p es la probabilidad de que un objeto pertenezca a la clase positiva. Penaliza los ejemplos positivos que están asociados con una baja probabilidad prevista y los ejemplos negativos que están asociados con una alta probabilidad prevista.

In [ ]:
dir(keras.activations)

### Así es como construimos una red neuronal completamente conectada en keras.




In [ ]:
model = Sequential()

# Agregue una capa de entrada y especifique su tamaño (número de características originales)

model.add(Dense(20, activation='relu', input_shape=(24,)))

# Agregue una capa oculta y especifique su tamaño

model.add(Dense(20, activation='relu'))

# Agregar una capa de salida

model.add(Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics = ['accuracy']) 

La palabra clave "métrica" aquí sirve para especificar otras posibles métricas que nos gustaría monitorear. La pérdida en sí no es interpretable, por lo que estaremos atentos a la precisión.

### ¿Listo para encajar?

¡Eso espero! Tenga en cuenta también los hiperparámetros adicionales "épocas" (el número de pasajes de ida y vuelta) y el tamaño del lote (cuántos de los datos se utilizan en cada paso en la actualización de los pesos).

In [ ]:
mynet = model.fit(X_train, y_train, validation_data= (X_val, y_val), epochs = 100,  batch_size=200)

Esto no se ve tan bien.

In [ ]:
plt.hist(model.predict(X_test), alpha = 0.5, label = 'pred')
plt.hist(y_test, alpha = 0.5, label = 'true')
plt.legend();

También es útil trazar las pérdidas de entrenamiento y validación a lo largo de las épocas.

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(121)

plt.plot(mynet.history['loss'], label = 'train')
plt.plot(mynet.history['val_loss'],'-.m', label = 'validation')
plt.ylabel('Loss', fontsize = 14)
plt.xlabel('Epoch', fontsize = 14)
plt.legend(loc='upper right', fontsize = 12)

plt.subplot(122)

plt.plot(mynet.history['accuracy'], label = 'train')
plt.plot(mynet.history['val_accuracy'], '-.m', label = 'validation')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Epoch', fontsize = 14)
plt.legend(fontsize = 12)
plt.subplots_adjust(wspace=0.5)

#plt.show()

#plt.savefig('FirstNN.png', dpi= 300)

### Revisión de aprendizaje
    
Mirando a los gráficos anteriores, ¿cómo diría que le está yendo a este clasificador? ¿Sufre de alta varianza o alto sesgo?

<br>

<details>
<summary style="display: list-item;">Haga clic aquí para la respuesta!</summary>
<p>
    
```
Los puntajes de entrenamiento y validación están cercanos, por lo que es un problema de alto sesgo, no de alta varianza. Esto se confirma por el hecho de que las notas son realmente bajas: alrededor del 70 % de precisión, en comparación con el > 90 % que obtuvimos con las SVM.
```

</p>
</details>

### Cuando algo sale mal, nuestro primer paso siempre debe ser volver a los fundamentos de la exploración/configuración de datos.

In [ ]:
X.describe()

### ¡Sí, nos olvidamos de escalar!

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

### Revisión de aprendizaje
    
Aplique el escalador anterior a la muestra correcta.

<br>

<details>
<summary style="display: list-item;">Haga clic aquí para la respuesta!</summary>
<p>

Como de costumbre, ¡solo usamos el conjunto de entrenamiento para derivar la escala! Necesitamos correr:

```python
scaler.fit(X_train)
```

</p>
</details>

In [ ]:
#¡Ejecute el código de la revisión de aprendizaje para continuar!


Ahora podemos usar el escalador ajustado para transformar los conjuntos de datos relevantes.



In [ ]:
Xst = scaler.transform(X)

In [ ]:
Xst.mean(axis=1) #Mosca: ¡no exactamente cero en todo el conjunto de datos!

In [ ]:
Xst_train = scaler.transform(X_train)
Xst_val = scaler.transform(X_val)
Xst_test = scaler.transform(X_test)

In [ ]:
mynet = model.fit(Xst_train, y_train, validation_data= (Xst_val, y_val), epochs=100, batch_size=200)

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(121)

plt.plot(mynet.history['loss'], label = 'train')
plt.plot(mynet.history['val_loss'],'-.m', label = 'validation')
plt.ylabel('Loss', fontsize = 14)
plt.xlabel('Epoch', fontsize = 14)
plt.legend(loc='upper right', fontsize = 12)

plt.subplot(122)

plt.plot(mynet.history['accuracy'], label = 'train')
plt.plot(mynet.history['val_accuracy'], '-.m', label = 'validation')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Epoch', fontsize = 14)
plt.legend(fontsize = 12)
plt.subplots_adjust(wspace=0.5)
#plt.show()

#plt.savefig('ScaledNN.png', dpi= 300)

### Revisión de aprendizaje
    
¿Cuál es su evaluación del clasificador anterior? 
<br>

<details><summary><b>¡Haga clic aquí para obtener la respuesta!</b></summary>
<p>
    
```
El rendimiento ahora es comparable al que habíamos obtenido con SVM. Hay indicios de alta varianza/sobreajuste, como lo muestra la brecha entre las notas del entrenamiento y la validación; es difícil saber cuán significativa es la brecha sin un enfoque de validación cruzada. También podemos ver que la pérdida de validación está aumentandose; esto indica que alguna técnica de regularización, como la detención anticipada y/o una capa de abandono, podría ayudar aquí.
```

In [ ]:
model = Sequential()

# Agregue una capa de entrada y especifique su tamaño (número de características originales)

model.add(Dense(20, activation='relu', input_shape=(24,)))

model.add(Dropout(0.2)) #Esta es la fracción de abandono

# Agregue una capa oculta y especifique su tamaño

model.add(Dense(20, activation='relu'))

model.add(Dropout(0.2)) #Esta es la fracción de abandono

# Agregar una capa de salida

model.add(Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics = ['accuracy']) 

#La palabra clave métrica aquí es para otras posibles métricas que nos gustaría monitorear

In [ ]:
mynet = model.fit(Xst_train, y_train, validation_data= (Xst_val, y_val), epochs=100, batch_size=200)

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(121)

plt.plot(mynet.history['loss'], label = 'train')
plt.plot(mynet.history['val_loss'],'-.m', label = 'validation')
plt.ylabel('Loss', fontsize = 14)
plt.xlabel('Epoch', fontsize = 14)
plt.legend(loc='upper right', fontsize = 12)

plt.subplot(122)

plt.plot(mynet.history['accuracy'], label = 'train')
plt.plot(mynet.history['val_accuracy'], '-.m', label = 'validation')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Epoch', fontsize = 14)
plt.legend(fontsize = 12)
plt.subplots_adjust(wspace=0.5)

#plt.savefig('RegularizedNN.png', dpi= 300)
#plt.show()

In [ ]:
#Evaluación final del modelo (mosca: esto se hace en el conjunto de prueba por lo que si hacemos optimización de parámetros en el pliegue de validación, este quedará afuera).

scores = model.evaluate(Xst_test, y_test, verbose=1)

print("Accuracy: %.2f%%" % (scores[1]*100)) #"scores" (notas) contiene la pérdida de prueba y la precisión, que estamos monitoreando

In [ ]:
scores